# Crawl dữ liệu

In [11]:
import os
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup

# Khởi tạo Cloudscraper (giả lập trình duyệt để vượt qua Cloudflare)
scraper = cloudscraper.create_scraper()

base_url = "https://www.kraken.com/vi-vn/prices?page="
all_data = []

# Lặp qua từng trang
for page in range(1, 5):  
    url = base_url + str(page)
    print(f"🔄 Đang tải trang {url}...")

    # Gửi request đến trang web
    response = scraper.get(url)

    # Kiểm tra nếu request bị chặn
    if response.status_code != 200:
        print(f"❌ Lỗi {response.status_code} khi tải trang {url}")
        continue  # Bỏ qua trang lỗi và thử trang tiếp theo

    # Phân tích HTML với BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Tìm tất cả các hàng dữ liệu <tr>
    rows = soup.find_all("tr", class_="fc150")
    
    for row in rows:
        cells = row.find_all("td")
        
        try:
            rank = cells[0].get_text(strip=True)  # Xếp hạng
            name = cells[1].find("span", class_="fc-bc02027a-6").get_text(strip=True)  # Tên coin
            symbol = cells[1].find("span", class_="fc-bc02027a-7").get_text(strip=True)  # Ký hiệu coin
            price = cells[2].get_text(strip=True)  # Giá hiện tại
            change_24h = cells[4].get_text(strip=True)  # Thay đổi 24h
            market_cap = cells[5].get_text(strip=True)  # Vốn hóa thị trường
            
            all_data.append([rank, name, symbol, price, change_24h, market_cap])
        
        except IndexError:
            continue  # Bỏ qua hàng lỗi nếu không đủ cột

# Tạo DataFrame và Lưu file CSV
columns = ["Xếp hạng", "Tên", "Mã", "Giá hiện tại", "Thay đổi 24h", "Vốn hóa thị trường"]
df = pd.DataFrame(all_data, columns=columns)

csv_path = os.path.join(os.getcwd(), "kraken_all_pages.csv")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"✅ Dữ liệu đã được lưu vào {csv_path}")

🔄 Đang tải trang https://www.kraken.com/vi-vn/prices?page=1...
🔄 Đang tải trang https://www.kraken.com/vi-vn/prices?page=2...
🔄 Đang tải trang https://www.kraken.com/vi-vn/prices?page=3...
🔄 Đang tải trang https://www.kraken.com/vi-vn/prices?page=4...
✅ Dữ liệu đã được lưu vào d:\E\2024.2\khdl (DS)\weekly\DataScience\weekly_exercise\ex1_crawlData\beautifulSoup\kraken\kraken_all_pages.csv


# Xuất ra một vài dữ liệu đầu tiên dưới dạng bảng

In [12]:
import pandas as pd
df = pd.read_csv("kraken_all_pages.csv", sep=",")
from IPython.display import display
display(df)

,Xếp hạng,Tên,Mã,Giá hiện tại,Thay đổi 24h,Vốn hóa thị trường
0,1,Bitcoin,BTC,"92.339,00 $","+6,09%","1,84 NT $"
1,2,Ethereum,ETH,"2.303,81 $","+5,79%","279,34 T $"
2,3,XRP,XRP,"2,52 $","+3,99%","146,59 T $"
3,4,Tether,USDT,"1,00 $","+0,02%","142,71 T $"
4,5,Solana,SOL,"149,11 $","+5,63%","76,04 T $"
...,...,...,...,...,...,...
352,353,Sky,SKY,"0,059 $","+3,45%",-
353,354,Multichain,MULTI,"0,86 $","-2,75%",-
354,355,Fantom,FTM,"0,60 $","+7,19%",-
355,356,Crust Shadow,CSM,"0,0056 $","+27,62%",-


Hiển thị vài dòng đầu tiên và thông tin chung của DataFrame

In [13]:
print("=== 5 dòng đầu tiên của dữ liệu ===")
print(df.head())
print("\n=== Thông tin dữ liệu ===")
df.info()

=== 5 dòng đầu tiên của dữ liệu ===
   Xếp hạng       Tên    Mã Giá hiện tại Thay đổi 24h Vốn hóa thị trường
0         1   Bitcoin   BTC  92.339,00 $       +6,09%          1,84 NT $
1         2  Ethereum   ETH   2.303,81 $       +5,79%         279,34 T $
2         3       XRP   XRP       2,52 $       +3,99%         146,59 T $
3         4    Tether  USDT       1,00 $       +0,02%         142,71 T $
4         5    Solana   SOL     149,11 $       +5,63%          76,04 T $

=== Thông tin dữ liệu ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Xếp hạng            357 non-null    int64 
 1   Tên                 357 non-null    object
 2   Mã                  357 non-null    object
 3   Giá hiện tại        357 non-null    object
 4   Thay đổi 24h        357 non-null    object
 5   Vốn hóa thị trường  357 non-null    object
dtypes: int